In [42]:
# Dependencies

#!pip install dimcli plotly tqdm -U --quiet
#!pip install openpyxl -U

In [50]:
import numpy as np
import sympy as sp
import pickle
from IPython.display import HTML
import pandas as pd
import itertools
import dimcli
import openpyxl
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import json



In [51]:
# Set up credentials and connection
dimcli.login(key="F86D0EE87BF7475E95D9A47574175BC3",
             endpoint="https://app.dimensions.ai")
dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.1)
Connected to: https://app.dimensions.ai - DSL v1.31
Method: manual login


Clean data

In [101]:
researchers = pd.read_excel("./Data/ethics-data/MASTER Ethics Register.xlsx", sheet_name="Register")
researchers = researchers[['Principal Investigator', 'CINSW Datasets']]

researchers = researchers.dropna().reset_index(drop = True)


remove_words = ['Dr', 'Professor', 'Mr', 'Associate', 'Prof', 'A/', 'Ms', '(contact Gemma Jacklyn)']
pat = r'\b(?:{})\b'.format('|'.join(remove_words))

researchers['cleanedPI'] = researchers['Principal Investigator'].str.replace(pat, '').str.strip()
researchers = researchers.drop_duplicates(subset='cleanedPI', keep="last").reset_index(drop=True)
researchers = researchers.drop(['Principal Investigator'], axis=1)

researchers.head(2)

Function to look up query:

In [180]:
def queryDimensions(lookupList = None, queryPrefix = None, querySuffix = None, returnConceptsFrame = False):
    noResults = []
    dataFromAPI = []
    for i in range(len(lookupList)):
        if returnConceptsFrame:
            try:
                r = dsl.query(queryPrefix + json.dumps(lookupList[i]) + querySuffix).as_dataframe_concepts()
            except:
                pass
        else:            
            r = dsl.query(queryPrefix + json.dumps(lookupList[i]) + querySuffix).as_dataframe()
        
        if (r.empty ):
            noResults.append(lookupList[i])
        else:
            r['entryFromLookupList'] = lookupList[i]
            dataFromAPI.append(r)
   
            dataFromAPIasSingleDataFrame = pd.concat(dataFromAPI)
        
    return({"dataFromAPI": dataFromAPIasSingleDataFrame, "noResults": noResults})

In [181]:
queryPrefix = "search publications where authors ="
querySuffix = " return publications[id+type+volume+year+issue+title+journal+authors + times_cited + concepts_scores] limit 300"

In [ ]:
# get publications
# publicationsAndCitations = queryDimensions(list(researchers.cleanedPI), queryPrefix, querySuffix, returnConceptsFrame=False)

In [209]:
# get concepts
# authorConcepts = queryDimensions(list(researchers.cleanedPI), queryPrefix, querySuffix, returnConceptsFrame=True)

In [211]:
# optional group concepts 
# authorConceptsGrouped = authorConcepts['dataFromAPI'].groupby(by=["entryFromLookupList", "concept"]).count()[['times_cited', 'title']]

In [208]:
publicationsAndCitations['dataFromAPI'].to_csv("./Data/publicationsAndCitations.csv")
authorConcepts['dataFromAPI'].to_csv("./Data/authoConceptsNotGrouped.csv")
authorConceptsGrouped.to_csv("./Data/authorConcepts.csv")researchers.head

In [217]:
authorConcepts['dataFromAPI']



,title,issue,id,type,times_cited,volume,authors,year,journal.id,journal.title,concepts_count,concept,score,frequency,score_avg,entryFromLookupList
2052,Abolishing the Tax‐Free Threshold in Australia...,2,pub.1052094549,article,1,30,[{'raw_affiliation': ['Department of Economics...,2009.0,jour.1043559,Fiscal Studies,55,tax-free threshold,0.960,2,0.8675,Guyonne Kalb
2753,Demographic and social-cognitive factors assoc...,NaN,pub.1130643516,article,3,39,[{'raw_affiliation': ['Charles Sturt Universit...,2020.0,jour.1028504,Eating Behaviors,49,social-cognitive factors,0.949,1,0.9490,John Attia
5365,Interplay between thermal convection and compr...,NaN,pub.1130159255,article,4,NaN,[{'raw_affiliation': ['State Key Laboratory of...,2020.0,jour.1136236,Mineralium Deposita,84,unconformity-related uranium deposits,0.946,2,0.8625,David Thomas


Function to calculate h-index for researchers

In [233]:
x = authorConcepts['dataFromAPI'].groupby(['entryFromLookupList', 'concept'])['score'].nlargest(4)

#x.reset_index(level=['entryFromLookupList', 'concept'])


#x.reset_index()
#authorConcepts['dataFromAPI'].nlargest(3,['score'])

x

KeyboardInterrupt: 

In [ ]:
def the_H_function(sorted_citations_list, n=1):
    """from a list of integers [n1, n2 ..] representing publications citations,
    return the max list-position which is >= integer

    eg
    >>> the_H_function([10, 8, 5, 4, 3]) => 4
    >>> the_H_function([25, 8, 5, 3, 3]) => 3
    >>> the_H_function([1000, 20]) => 2
    """
    if sorted_citations_list and sorted_citations_list[0] >= n:
        return the_H_function(sorted_citations_list[1:], n+1)
    else:
        return n-1

def get_pubs_citations(researcher_id):
    q = """search publications where researchers.id = "{}" return publications[times_cited] sort by times_cited limit 1000"""
    pubs = dsl.query(q.format(researcher_id))
    return list(pubs.as_dataframe().fillna(0)['times_cited'])


print("H_index is:", the_H_function(get_pubs_citations(RESEARCHER)))